In [1]:
import sqlite3
import pandas as pd
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
import os

import plotly.graph_objs as go
import sys
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
# helper function to set up connection to database.
def _conn_db(path):
    database = os.path.join(path, 'database.sqlite')
    conn = sqlite3.connect(database)
    return conn
# helper function to get the data.
def _get_table(query,conn):
    df = pd.read_sql(query,conn)
    return df

In [3]:
path="."
conn = _conn_db(path)

I am gonna draw a stacked bar plot.  
I want to focus on Premier League, and see the composition of goals scored for the all teams over all goals scored in that season.  
  
Plan: 
1. get the data.
    1. Get the table that contains matches with leagues name.
    2. Compute the total goal scored by summing the home team goal and away team goal.
    3. Compute each team's goal scored [Unclear!]
    4. Make a stacked bar graph.
        1. X will be the season name.
        2. Y will be each team's goal scored in that season.

In [23]:
query = "select r3.name as League_name, r.team_long_name as home_team_name1, \
        r.team_short_name as home_team_name2,r2.team_long_name as away_team_name1, r2.team_short_name as \
        away_team_name2,l.season,l.home_team_goal,l.away_team_goal from Match as l left join Team as r \
        on l.home_team_api_id = r.team_api_id \
        left join Team as r2 \
        on l.away_team_api_id=r2.team_api_id\
        left join League as r3\
        on l.league_id = r3.id;"
match_with_team_name_df = _get_table(query, conn)

In [31]:
query = "select * from League;"
league_df = _get_table(query, conn)
league_name = league_df.name

In [50]:
season_ls = ["2008/2009","2009/2010","2010/2011","2011/2012","2012/2013","2013/2014","2014/2015","2015/2016"]
league_name[1]

'England Premier League'

In [113]:
def _get_league_score_on_year(league_name, season):
    """Get the specified leagues's total games on certain season.""" 
    # get table with team name along with home goal and away goal.
    query = "select r3.name as League_name, r.team_long_name as home_team_name1, \
        r.team_short_name as home_team_name2,r2.team_long_name as away_team_name1, r2.team_short_name as \
        away_team_name2,l.season,l.home_team_goal,l.away_team_goal from Match as l left join Team as r \
        on l.home_team_api_id = r.team_api_id \
        left join Team as r2 \
        on l.away_team_api_id=r2.team_api_id\
        left join League as r3\
        on l.league_id = r3.id;"
    df = _get_table(query, conn)
    # get all matches in one season for one league.
    res_df = df[(df.League_name == league_name) & (df.season == season)]
    # get all goals scored in home and away team.
    all_goals = [sum(res_df.home_team_goal),sum(res_df.away_team_goal)]
    # get individual teams goal
    teams_goals_df = res_df.groupby(by = "home_team_name1").sum()[["home_team_goal","away_team_goal"]]
    teams_goals_df["tot_goals"] = teams_goals_df.home_team_goal + teams_goals_df.away_team_goal
    top_4_home_teams = teams_goals_df.sort_values(by="tot_goals",ascending=False).head(4)
    return top_4_home_teams

In [114]:
df = _get_league_score_on_year(league_name[1], season_ls[7])

To draw the top four best home team's goals scored over the years. And their away games' goal scored for years over 2008/2009 to 2015/2016.

In [133]:
def _plot_top_attact_team(league_name):
    """
    Get the plot for each leagues' top 4 attacking teams goals stat over the years.
    param:
        league_name: the name of the league.
    return:
        fig: object to draw.
    """
    data = []
    # build up traces.
    for season in season_ls:
        # get the data
        df = _get_league_score_on_year(league_name, season)
        trace = go.Bar(
                x=df.index.values,
                y=(df.home_team_goal + df.away_team_goal).values,
                name=season
                )
        data.append(trace)
        
        layout = go.Layout(
        barmode='stack',
        title="Top 4 Attacking Teams On Each Season in {}".format(league_name),
        xaxis=dict(
            title='Team Name',
            titlefont=dict(
                family='Old Standard TT, serif',
                size=14,
                color='black'
            )
        ),
        yaxis=dict(
            title='Number of Goals',
            titlefont=dict(
                family='Old Standard TT, serif',
                size=18,
                color='black'
            )
        )
    )

    fig = go.Figure(data=data, layout=layout)
    return fig

In [140]:
fig = _plot_top_attact_team(league_name[7])

In [141]:
iplot(fig)

In [142]:
# make html file.
html_doc = '''<!DOCTYPE html>
<html>
<head>
  <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
</head>

<div>
     <div style="display:flex">
     <h2><font color="black">EU Soccer Top Attacking Teams on Each League Goals Scored Info.</font></h2>
     </div>
</div>
'''

div_ls = []
for i in league_name:
    fig = _plot_top_attact_team(i)
    _div = plot(fig, include_plotlyjs=False, output_type='div')
    html_doc += '''<h5><a id="1"></a>%(_div_title)s</h5> 
                      %(div)s
                ''' % {'div':_div, '_div_title':i+"'s Goal Scored Stat."}
    
html_doc += '''<hr>'''
html_doc +='''</body>
                  </html>'''


In [143]:
with open('EU_soccer_team_scores.html', 'w') as f:
    f.write(html_doc)

In [144]:
!open EU_soccer_team_scores.html